### 10x genomics 플랫폼의 종류

| | **Chromium Single Cell** | **Visium Spatial** | **Xenium In Situ** |
| :--- | :--- | :--- | :--- |
| **Platform** | Chromium | Visium| Xenium|
| **When to use** | **종합적인 단일 세포 데이터**<br>세포 집단과 상태에 대한 심층 분석에 이상적 | **고해상도 공간 유전자 발현**<br>복잡한 조직, 이웃 세포, 세포 간 상호작용 이해.<br>다른 공간 오믹스, 조직학, 형태학과 통합. | Visium 과 동일 |
| **Why to use** | 편향 없는 단일 세포 발견,<br>유전자별 높은 민감도 | 편향 없는 공간적 발견 | 표적 기반 공간 탐색,<br>유전자별 높은 민감도 |
| **Applications** | 전체 전사체 유전자 발현<br>단백질<br>TCR, BCR<br>CRISPR<br>ATAC | 전체 전사체 유전자 발현 | 표적 유전자 발현 (최대 5,000개 유전자) |
| **Resolution** | 단일 세포 (Single cell) | 2-µm 영역에 할당된 전사체 | 단일 세포 (Single cell) |
| **Data readout** | Sequencing 기반 | Sequencing 기반 | Image 기반 |
| **Sample compatibility**| Single cell or nuclei suspensions from fresh, frozen, or FFPE samples | FFPE<br>Fresh frozen<br>Fixed frozen | Fresh frozen<br>FFPE |

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
from scipy.sparse import csr_matrix, isspmatrix

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns

#ultils
#import ipynbname
from datetime import datetime

In [3]:
# Custom functions
sys.path.append('HelperFunctions')
import Day1Helper as fn
#help(fn)

In [ ]:
input_file = '../1_AssembledAdata.h5ad'

In [ ]:
#Cell filtering   
MIN_GENES = 850
MAX_GENES = 10000

MIN_COUNTS = 1500
MAX_COUNTS = 15000

PCT_MITO = 10
PCT_RIBO = 25

#Gene Filtering
PCT_CELLS = 0.25

**미토콘드리아 비율을 확인하는 이유**
> "A high fraction of mitochondrial counts can be indicative of cells with broken membranes. In such cases, the cytoplasmic mRNA has leaked out, leaving behind the more conserved mRNA located in the mitochondria."    
>
> _높은 비율의 미토콘드리아 유전자 발현량은 세포막이 손상된 세포를 나타내는 지표가 될 수 있습니다. 이러한 경우, 세포질의 mRNA는 밖으로 유출되고 미토콘드리아 내의 보존되어있는 mRNA만 남게 됩니다._
>
>_Current best practices in single‐cell RNA‐seq analysis: a tutorial (2019)_

    

**리보솜 유전자 비율을 확인하는 이유**
>"A large number of studies have shown that ribosomal protein-coding genes (RPGs) are highly expressed and co-expressed in many cell types. [...]    
>the expression of RPGs is often correlated with the cell cycle, and in some studies, RPGs are removed before downstream analysis. [...]    
>However, the expression of RPGs can also reflect the metabolic activity of cells, and their expression levels are different in different cell types."
>
>_많은 연구에서 리보솜 단백질 암호화 유전자(RPG)가 다양한 세포 유형에서 높게 발현되고 함께 발현된다는 것을 보여주었습니다. [...]    
>RPG의 발현은 종종 **세포 주기**와 연관되어 있으며, 일부 연구에서는 다음 분석 단계 전에 **RPG를 제거**하기도 합니다. [...]    
>하지만, RPG의 발현은 세포의 대사 활동을 반영할 수도 있으며, 그 발현 수준은 다른 세포 유형에서 다르게 나타납니다._
>
>_Effective ribosomal RNA depletion for single-cell total RNA-seq by scDASH (2021)_

In [ ]:
# batch key for calculation of HVG and for Harmony correction 
BATCH_KEY = 'batch_key'

In [4]:
# Number of Principal component for UMAP
N_PCs = 18

#PCA 분석 -> 차원축소 이후 상위 18개의 주성분만 UMAP 계산에 사용

In [5]:
# Leiden resolution chosen for clusters
chosen_leiden = 'Leiden_04'

#세포 군집을 위해 사용하는 Leiden 알고리즘을 해상도 0.4로 실행

### Anndata에 대하여
<img src="https://raw.githubusercontent.com/scverse/anndata/main/docs/_static/img/anndata_schema.svg" width="400">    

scRNA-seq 데이터를 다루기 위해 만들어진 데이터 구조    
* adata.X: 핵심 데이터셋. 행은 세포, 열은 유전자. 희소행렬/밀집행렬을 이용한 메모리효율적 구조로 이루어짐.
* adata.obs: 세포에 대한 메타데이터. 각 행은 adata.X의 행과 같음. Sequencing에 따른 세포유형, 실험배치, QC matrics 등을 가짐
* adata.var: 유전자에 대한 메타데이터. 각 행은 adata.X의 열과 같음. 유전자 ID 등을 가짐
* adata.obsm: 세포에 대한 다차원 데이터.
* adata.varm: 유전자에 대한 다차원 데이터.
* adata.obsp: 세포-세포 사이의 관계를 나타내는 그래프 데이터.
* adata.varp: 유전자-유전사 사이의 관계를 나타내는 그래프 데이터.
* adata.layers: adata.X에 담지 못한 추가 데이터
* adata.uns: 비정형 데이터셋

### Barcode metadata
* Barcode = 각 세포들에게 부여된 ID

In [ ]:
barcodes = adata.obs.index